In [1]:
%config Completer.use_jedi = False

# Prepearing Data

# Read

In [2]:
import pandas as pd


In [4]:
all_data=pd.read_csv("../data/Data.csv")
all_data=all_data.drop(columns=['base_url','likes'])
all_data['sumreactions']=all_data['love']+all_data['sigh']+all_data['wow']+all_data['care']+all_data['grrr']
all_data

,love,base_name,sigh,grrr,category,wow,care,content,sumreactions
0,5,سوريا اليوم,2,1,Social,0,0,جديد قضية ابن بايدن.. الأف بي آي حقق مع شريكه ...,8
1,0,قناة العربية Al Arabiya,0,0,Social,0,0,#هاااام : بمتابعة واشراف من عمر حورية أمين ال...,0
2,1,شام تايمز shamtimes.net,0,0,Military,0,0,وَلَا تَحْسَبَنَّ اللَّهَ غَافِلًا عَمَّا يَعْ...,1
3,27,شبكة أخبار اللاذقية L.N.N,0,0,Religion,0,3,مسافرة أسترالية للعربية: تم اقتياد النساء والأ...,30
4,0,الياطر نيوز Alyater News,0,0,Politics,1,0,Trump in 2016: “#Assad is killing #ISIS. #Russ...,1
...,...,...,...,...,...,...,...,...,...
29942,0,المركز الاذاعي والتلفزيوني في السويداء,0,0,Education,0,0,#بث #مباشر - #برنامج #حق_المواطن_في_الإعلام ض...,0
29943,48,تلفزيون اللاذقية _ Lattakia TV,72,0,Military,1,5,وتمضي الأيام والباسل في الذاكرة والوجدان ❤ فار...,126
29944,2,وزارة الاتصالات والتقانة - سورية,0,0,Politics,0,0,رئاسة مجلس الوزراء في سورية وزارة الاتصالات و...,2
29945,1,الحرس الجمهوري السوري الالكتروني اللواء 105,0,0,Military,0,0,نائب رئيس المركز الروسي للمصالحة في سوريا: ناق...,1


# Clean

In [5]:
df_age_negative = all_data[ all_data['sumreactions'] < 1 ] # Step 1
all_data = all_data.drop(df_age_negative.index, axis=0) 

# Clean content

In [8]:
import emoji
import functools
import operator
import re
import pandas as pd


def split_emoji(txt):
    x=' '.join(c for c in txt.split() if c not in emoji.UNICODE_EMOJI)
    return x
#     em = txt
#     em_split_emoji = emoji.get_emoji_regexp().split(em)
#     em_split_whitespace = [substr.split() for substr in em_split_emoji]
#     em_split = functools.reduce(operator.concat, em_split_whitespace)
#     res = ''

#     for i in range(len(em_split)):
#         if em_split[i] not in emoji.UNICODE_EMOJI:
#             if i != len(em_split) - 1:
#                 res += em_split[i] + ' '
#             else:
#                 res += em_split[i]
#         else:
#             continue
#     return res

stop_word=pd.read_csv("../data/stopwords.csv",names=['data'])['data'].values
i=0
def len_sent(text):
    return len(text.split())
def preprocessing(data ):
    data=str(data)
    data = split_emoji(data)
    data = str(' '.join(re.sub("([٠١٢٣٤٥٦٧٨٩]+)|([0-9]+)|([A-Za-z]+)|\_+|(\#)+|(\/)+|(\:)+", " ", data).split()))
    data = re.sub("[إأٱآا]", "ا", data)
    data = re.sub("ة", "ه", data)
    # remove duplicate
    noise = re.compile(""" ّ    | # Tashdid
                            َ    | # Fatha
                            ً    | # Tanwin Fath
                            ُ    | # Damma
                            ٌ    | # Tanwin Damm
                            ِ    | # Kasra
                            ٍ    | # Tanwin Kasr
                            ْ    | # Sukun
                            ـ   |  # Tatwil/Kashida
                            ،   |
                        """, re.VERBOSE)
    flagsUs = re.compile("["
                         u"\U0001F680-\U0001F6FF"  # transport & map symbols
                         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                         "]+", flags=re.UNICODE)
    dirtyChars = re.compile("["
                            "\u0600-\u0620"
                            "\u063B-\u0640"
                            "\u064B-\u065F"
                            "\u066A-\u06FF"
                            "\u0750-\u077F"
                            "\u08A0-\u08FF"
                            "\uFB50-\uFBE9"
                            "\uFBF0-\uFBFB"
                            "\uFC5B-\uFC63"
                            "\uFCF2-\uFCF4"
                            "\uFD3C-\uFD4F"
                            "\uFD90-\uFD91"
                            "\uFDC8-\uFDFF"
                            "\uFE70-\uFE7F"
                            "\uFEFD-\uFEFF"
                            "]+", flags=re.UNICODE)
    data = str(re.sub(flagsUs, '', data))
    data = str(re.sub(dirtyChars, '', data))
    data = str(re.sub(noise, '', data))
#     data = str(''.join(c for c in data if c not in punctuation))
    data = re.sub(
        '\/+|\●+|\◽+|\٪+|\▪+|\»+|\«+|\_+|\ʚïɞ+|\▐+|\►+|\"+|\*+|\▁+|\》+|\《+|\[+|\Ещё+|\]+|\|+|\;+|\'+|\<+|\>+|\\+|\`+|\{+|\}+|\~+|\"+|\-+|\:+|\@+|\#+|\$+|\ﷺ+|\%+|\^+|\&+|\(+|\)+|\.+|\,+|\?+|\=+|\++|\؛+\“+|\”+',
        ' ', data)
    data = re.sub('\!', '  ', data)
    data = re.sub('\⚘', '  ', data)
    data = re.sub('\��', '  ', data)
    data = re.sub('\؟ ', ' ', data)
    data = re.sub('\.', ' ', data)
    data = re.sub('\s+', ' ', data)
    data = re.sub('\\\+', ' ', data)
    data = re.sub('\"+',' ',data)
    data = re.sub(' ا ', ' ', data)
    data = re.sub(' ب ', ' ', data)
    data = re.sub(' ت ', ' ', data)
    data = re.sub(' ث ', ' ', data)
    data = re.sub(' ج ', ' ', data)
    data = re.sub(' ح ', ' ', data)
    data = re.sub(' خ ', ' ', data)
    data = re.sub(' د ', ' ', data)
    data = re.sub(' ذ ', ' ', data)
    data = re.sub(' ر ', ' ', data)
    data = re.sub(' ز ', ' ', data)
    data = re.sub(' س ', ' ', data)
    data = re.sub(' ش ', ' ', data)
    data = re.sub(' ص ', ' ', data)
    data = re.sub(' ض ', ' ', data)
    data = re.sub(' ط ', ' ', data)
    data = re.sub(' ظ ', ' ', data)
    data = re.sub(' ع ', ' ', data)
    data = re.sub(' غ ', ' ', data)
    data = re.sub(' ف ', ' ', data)
    data = re.sub(' ق ', ' ', data)
    data = re.sub(' ك ', ' ', data)
    data = re.sub(' ل ', ' ', data)
    data = re.sub(' م ', ' ', data)
    data = re.sub(' ن ', ' ', data)
    data = re.sub(' ه ', ' ', data)
    data = re.sub(' و ', ' ', data)
    data = re.sub(' ي ', ' ', data)
    data = re.sub(' ئ ', ' ', data)
    data = re.sub(' ؤ ', ' ', data)
    data = re.sub(' ء ', ' ', data)
    data = re.sub('\_+', ' ', data)
    data = re.sub('\…+', ' ', data)
    data = re.sub('\“|\”', '', data)
    data = re.sub(r'([\u0600-\u06FF])\1{3,}', r'\1\1\1', data)
    data = re.sub(r'[\u2066]', ' ', data)
    data = re.sub(r'[\u2069]', ' ', data)
    data = re.sub(r'[\uFE0F]', ' ', data)
    data = re.sub(r'[\u25a0]', ' ', data)
    data = re.sub(r'[\u2022]', ' ', data)
    data = re.sub(r'[\u2592]', ' ', data)
    data = re.sub('[\u1ea0]|[\u1ea1]|[\u1e97]|[\u1ea1]|[\u02bf]|[\u1e97]|[\u1ea1]|[\u1e97]|[\u1ea1]|[\u1ea1]|[\u02be]|[\u1ea1]|[\u1ea1]',' ',data)
    data = ' '.join([word for word in data.split() if word not in stop_word])
    data = " ".join(data.split())
    return data


In [9]:
all_data['contentclean']=all_data['content'].apply(preprocessing)

In [10]:
all_data['content']

0        جديد قضية ابن بايدن.. الأف بي آي حقق مع شريكه ...
2        وَلَا تَحْسَبَنَّ اللَّهَ غَافِلًا عَمَّا يَعْ...
3        مسافرة أسترالية للعربية: تم اقتياد النساء والأ...
4        Trump in 2016: “#Assad is killing #ISIS. #Russ...
5        قوانين هذا العالم العامة مثل قواعد الحياة ذاته...
                               ...                        
29940    من نص ساعة طفيت الكهربا وهلق رجعت اجت مع العلم...
29941    📶#الباقة_التوفيرية_الجديدة📶  3 غيغابايت سيرف +...
29943    وتمضي الأيام والباسل في الذاكرة والوجدان ❤ فار...
29944    رئاسة مجلس الوزراء في سورية  وزارة الاتصالات و...
29945    نائب رئيس المركز الروسي للمصالحة في سوريا: ناق...
Name: content, Length: 21693, dtype: object

In [11]:
all_data.to_csv("../data/Data_cleaning_with_content.csv",index=False)